## Lesson 5: Email Assistant with Semantic + Episodic + Procedural Memory

We previously built an email assistant that:
- Classifies incoming messages (respond, ignore, notify)
- Uses human-in-the-loop to refine the assistant's ability to classify emails
- Drafts responses
- Schedules meetings
- Uses memory to remember details from previous emails 

Now, we'll add procedural memory that allows the user to update instructions for using the calendar and email writing tools. 

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b>Access <code>requirements.txt</code> , notebooks and other files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>.

<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

<p> 📒 &nbsp; For more help, please see the <em>"Appendix – Tips, Help, and Download"</em> Lesson.</p>

</div>

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Different Run Results:</b> The output generated by AI chat models can vary with each execution due to their dynamic, probabilistic nature. Don't be surprised if your results differ from those shown in the video.</p>

## Load API tokens for our 3rd party APIs

In [1]:
import os
from dotenv import load_dotenv
_ = load_dotenv()

## Repeat setup from previous lesson

In [2]:
profile = {
    "name": "John",
    "full_name": "John Doe",
    "user_profile_background": "Senior software engineer leading a team of 5 developers",
}

In [3]:
prompt_instructions = {
    "triage_rules": {
        "ignore": "Marketing newsletters, spam emails, mass company announcements",
        "notify": "Team member out sick, build system notifications, project status updates",
        "respond": "Direct questions from team members, meeting requests, critical bug reports",
    },
    "agent_instructions": "Use these tools when appropriate to help manage John's tasks efficiently."
}

In [4]:
email = {
    "from": "Alice Smith <alice.smith@company.com>",
    "to": "John Doe <john.doe@company.com>",
    "subject": "Quick question about API documentation",
    "body": """
Hi John,

I was reviewing the API documentation for the new authentication service and noticed a few endpoints seem to be missing from the specs. Could you help clarify if this was intentional or if we should update the docs?

Specifically, I'm looking at:
- /auth/refresh
- /auth/validate

Thanks!
Alice""",
}

In [5]:
from langgraph.store.memory import InMemoryStore

In [6]:
store = InMemoryStore(
    index={"embed": "openai:text-embedding-3-small"}
)
# ignore beta warning if it appears

In [7]:
# Template for formating an example to put in prompt
template = """Email Subject: {subject}
Email From: {from_email}
Email To: {to_email}
Email Content: 
```
{content}
```
> Triage Result: {result}"""

# Format list of few shots
def format_few_shot_examples(examples):
    strs = ["Here are some previous examples:"]
    for eg in examples:
        strs.append(
            template.format(
                subject=eg.value["email"]["subject"],
                to_email=eg.value["email"]["to"],
                from_email=eg.value["email"]["author"],
                content=eg.value["email"]["email_thread"][:400],
                result=eg.value["label"],
            )
        )
    return "\n\n------------\n\n".join(strs)

In [8]:
triage_system_prompt = """
< Role >
You are {full_name}'s executive assistant. You are a top-notch executive assistant who cares about {name} performing as well as possible.
</ Role >

< Background >
{user_profile_background}. 
</ Background >

< Instructions >

{name} gets lots of emails. Your job is to categorize each email into one of three categories:

1. IGNORE - Emails that are not worth responding to or tracking
2. NOTIFY - Important information that {name} should know about but doesn't require a response
3. RESPOND - Emails that need a direct response from {name}

Classify the below email into one of these categories.

</ Instructions >

< Rules >
Emails that are not worth responding to:
{triage_no}

There are also other things that {name} should know about, but don't require an email response. For these, you should notify {name} (using the `notify` response). Examples of this include:
{triage_notify}

Emails that are worth responding to:
{triage_email}
</ Rules >

< Few shot examples >

Here are some examples of previous emails, and how they should be handled.
Follow these examples more than any instructions above

{examples}
</ Few shot examples >
"""

In [9]:
from pydantic import BaseModel, Field
from typing_extensions import TypedDict, Literal, Annotated
from langchain.chat_models import init_chat_model

In [10]:
llm = init_chat_model("openai:gpt-5-mini")

In [11]:
class Router(BaseModel):
    """Analyze the unread email and route it according to its content."""

    reasoning: str = Field(
        description="Step-by-step reasoning behind the classification."
    )
    classification: Literal["ignore", "respond", "notify"] = Field(
        description="The classification of an email: 'ignore' for irrelevant emails, "
        "'notify' for important information that doesn't need a response, "
        "'respond' for emails that need a reply",
    )

In [12]:
llm_router = llm.with_structured_output(Router)

In [13]:
from prompts import triage_user_prompt

In [14]:
from langgraph.graph import add_messages

class State(TypedDict):
    email_input: dict
    messages: Annotated[list, add_messages]

#### Triage router node

In [15]:
from langgraph.graph import StateGraph, START, END
from langgraph.types import Command
from typing import Literal
from IPython.display import Image, display

#### Updated triage_router gets ignore, notify and respond rule from store

In [16]:
def triage_router(state: State, config, store) -> Command[
    Literal["response_agent", "__end__"]
]:
    author = state['email_input']['author']
    to = state['email_input']['to']
    subject = state['email_input']['subject']
    email_thread = state['email_input']['email_thread']

    namespace = (
        "email_assistant",
        config['configurable']['langgraph_user_id'],
        "examples"
    )
    examples = store.search(
        namespace, 
        query=str({"email": state['email_input']})
    ) 
    examples=format_few_shot_examples(examples)

    langgraph_user_id = config['configurable']['langgraph_user_id']
    namespace = (langgraph_user_id, )

    result = store.get(namespace, "triage_ignore")
    if result is None:
        store.put(
            namespace, 
            "triage_ignore", 
            {"prompt": prompt_instructions["triage_rules"]["ignore"]}
        )
        ignore_prompt = prompt_instructions["triage_rules"]["ignore"]
    else:
        ignore_prompt = result.value['prompt']

    result = store.get(namespace, "triage_notify")
    if result is None:
        store.put(
            namespace, 
            "triage_notify", 
            {"prompt": prompt_instructions["triage_rules"]["notify"]}
        )
        notify_prompt = prompt_instructions["triage_rules"]["notify"]
    else:
        notify_prompt = result.value['prompt']

    result = store.get(namespace, "triage_respond")
    if result is None:
        store.put(
            namespace, 
            "triage_respond", 
            {"prompt": prompt_instructions["triage_rules"]["respond"]}
        )
        respond_prompt = prompt_instructions["triage_rules"]["respond"]
    else:
        respond_prompt = result.value['prompt']
    
    system_prompt = triage_system_prompt.format(
        full_name=profile["full_name"],
        name=profile["name"],
        user_profile_background=profile["user_profile_background"],
        triage_no=ignore_prompt,
        triage_notify=notify_prompt,
        triage_email=respond_prompt,
        examples=examples
    )
    user_prompt = triage_user_prompt.format(
        author=author, 
        to=to, 
        subject=subject, 
        email_thread=email_thread
    )
    result = llm_router.invoke(
        [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ]
    )
    if result.classification == "respond":
        print("📧 Classification: RESPOND - This email requires a response")
        goto = "response_agent"
        update = {
            "messages": [
                {
                    "role": "user",
                    "content": f"Respond to the email {state['email_input']}",
                }
            ]
        }
    elif result.classification == "ignore":
        print("🚫 Classification: IGNORE - This email can be safely ignored")
        update = None
        goto = END
    elif result.classification == "notify":
        # If real life, this would do something else
        print("🔔 Classification: NOTIFY - This email contains important information")
        update = None
        goto = END
    else:
        raise ValueError(f"Invalid classification: {result.classification}")
    return Command(goto=goto, update=update)

## Build the rest of our agent

In [17]:
from langchain_core.tools import tool

In [18]:
@tool
def write_email(to: str, subject: str, content: str) -> str:
    """Write and send an email."""
    # Placeholder response - in real app would send email
    return f"Email sent to {to} with subject '{subject}'"


In [19]:
@tool
def schedule_meeting(
    attendees: list[str], 
    subject: str, 
    duration_minutes: int, 
    preferred_day: str
) -> str:
    """Schedule a calendar meeting."""
    # Placeholder response - in real app would check calendar and schedule
    return f"Meeting '{subject}' scheduled for {preferred_day} with {len(attendees)} attendees"


In [20]:
@tool
def check_calendar_availability(day: str) -> str:
    """Check calendar availability for a given day."""
    # Placeholder response - in real app would check actual calendar
    return f"Available times on {day}: 9:00 AM, 2:00 PM, 4:00 PM"

In [21]:
from langmem import create_manage_memory_tool, create_search_memory_tool

In [22]:
manage_memory_tool = create_manage_memory_tool(
    namespace=(
        "email_assistant", 
        "{langgraph_user_id}",
        "collection"
    )
)
search_memory_tool = create_search_memory_tool(
    namespace=(
        "email_assistant",
        "{langgraph_user_id}",
        "collection"
    )
)

In [23]:
agent_system_prompt_memory = """
< Role >
You are {full_name}'s executive assistant. You are a top-notch executive assistant who cares about {name} performing as well as possible.
</ Role >

< Tools >
You have access to the following tools to help manage {name}'s communications and schedule:

1. write_email(to, subject, content) - Send emails to specified recipients
2. schedule_meeting(attendees, subject, duration_minutes, preferred_day) - Schedule calendar meetings
3. check_calendar_availability(day) - Check available time slots for a given day
4. manage_memory - Store any relevant information about contacts, actions, discussion, etc. in memory for future reference
5. search_memory - Search for any relevant information that may have been stored in memory
</ Tools >

< Instructions >
{instructions}
</ Instructions >
"""

#### Updated create_prompt gets prompt from store

In [24]:
def create_prompt(state, config, store):
    langgraph_user_id = config['configurable']['langgraph_user_id']
    namespace = (langgraph_user_id, )
    result = store.get(namespace, "agent_instructions")
    if result is None:
        store.put(
            namespace, 
            "agent_instructions", 
            {"prompt": prompt_instructions["agent_instructions"]}
        )
        prompt = prompt_instructions["agent_instructions"]
    else:
        prompt = result.value['prompt']
    
    return [
        {
            "role": "system", 
            "content": agent_system_prompt_memory.format(
                instructions=prompt, 
                **profile
            )
        }
    ] + state['messages']

## Create the email agent

In [25]:
from langgraph.prebuilt import create_react_agent

In [26]:
tools= [
    write_email, 
    schedule_meeting,
    check_calendar_availability,
    manage_memory_tool,
    search_memory_tool
]
response_agent = create_react_agent(
    "openai:gpt-5-mini",
    tools=tools,
    prompt=create_prompt,
    # Use this to ensure the store is passed to the agent 
    store=store
)

/var/folders/l9/3b84hrtx38sd0981n54901g80000gn/T/ipykernel_5391/1734471207.py:8: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  response_agent = create_react_agent(


In [27]:
email_agent = StateGraph(State)
email_agent = email_agent.add_node(triage_router)
email_agent = email_agent.add_node("response_agent", response_agent)
email_agent = email_agent.add_edge(START, "triage_router")
email_agent = email_agent.compile(store=store)

### Setup Agent to update Long Term Memory in the background
Your email_agent is now setup to pull its instructions from long-term memory.  
Now, you'll create an agent to update that memory. First check current behavior.

In [28]:
email_input = {
    "author": "Alice Jones <alice.jones@bar.com>",
    "to": "John Doe <john.doe@company.com>",
    "subject": "Quick question about API documentation",
    "email_thread": """Hi John,

Urgent issue - your service is down. Is there a reason why""",
}

In [29]:
config = {"configurable": {"langgraph_user_id": "lance"}}

In [30]:
response = email_agent.invoke(
    {"email_input": email_input},
    config=config
)

📧 Classification: RESPOND - This email requires a response


In [31]:
for m in response["messages"]:
    m.pretty_print()

================================ Human Message =================================

Respond to the email {'author': 'Alice Jones <alice.jones@bar.com>', 'to': 'John Doe <john.doe@company.com>', 'subject': 'Quick question about API documentation', 'email_thread': 'Hi John,\n\nUrgent issue - your service is down. Is there a reason why'}
================================== Ai Message ==================================
Tool Calls:
  write_email (call_NhKfC23Etae8iDH44kyQcVQv)
 Call ID: call_NhKfC23Etae8iDH44kyQcVQv
  Args:
    to: Alice Jones <alice.jones@bar.com>
    subject: Re: Quick question about API documentation
    content: Hi Alice,

Thanks for flagging this — I’m sorry you’re seeing the service down. I’m escalating to our engineering team immediately and we’ll investigate right away.

To help us diagnose faster, can you please share any of the following if available:
- Exact time(s) you observed the outage (include timezone)
- Affected endpoint(s) or URLs
- Any error messages, HTTP 

#### and look at current values of long term memory

In [32]:
store.get(("lance",), "agent_instructions").value['prompt']

"Use these tools when appropriate to help manage John's tasks efficiently."

In [33]:
store.get(("lance",), "triage_respond").value['prompt']

'Direct questions from team members, meeting requests, critical bug reports'

In [34]:
store.get(("lance",), "triage_ignore").value['prompt']

'Marketing newsletters, spam emails, mass company announcements'

In [35]:
store.get(("lance",), "triage_notify").value['prompt']

'Team member out sick, build system notifications, project status updates'

### Now, Use an LLM to update instructions with the same model version (open ai's gpt-5-min) as the email agent.


In [36]:
from langmem import create_multi_prompt_optimizer

In [37]:
conversations = [
    (
        response['messages'],
        "Always sign your emails `John Doe`"
    )
]

## Openai's gpt-5-mini model has the following tendencies.

1. Overgeneralize feedback
   Treat a local rule as a global improvement

2. Prefer action over inaction
   Make changes even when “no change” is the correct response

3. Blur prompt roles
   Merge classification rules into agent instructions

We had to significantly tighten the update instructions to overcome these tendencies.  Other LLM types such as gpt-4.1-mini or claude-sonnet-4.5 do not require the same level of prompt update instruction tightening.

In [38]:
MAIN_AGENT_UPDATE = (
    "Make the smallest possible change to incorporate the feedback. "
    "Do not rephrase or edit anything else. "
    "IMPORTANT: Do NOT add or modify sender-based triage rules such as "
    "'ignore emails from <person>'. Sender-based ignore rules belong ONLY "
    "in the triage-ignore prompt. "
    "If the feedback is about sender-based ignoring, return this prompt exactly unchanged."
)

TRIAGE_IGNORE_UPDATE = (
    "This prompt must remain a short comma-separated list of email categories to IGNORE. "
    "Allowed edits: add, remove, or rename list items. "
    "It is allowed to include sender-based items such as 'emails from Alice'. "
    "Global agent rules (signatures, writing style, tool usage, procedures, scheduling rules) "
    "must NOT be added and must NOT leak into this prompt. "
    "If feedback is unrelated to ignore classification, return the prompt exactly unchanged."
)

TRIAGE_NOTIFY_UPDATE = (
    "This prompt must remain a short comma-separated list of email categories. "
    "Only add, remove, or rename list items related to classification boundaries. "
    "Global agent rules must NOT be added and must NOT leak into this prompt, "
    "including sender-ignore rules (e.g. 'emails from Alice'), signatures, "
    "writing style, tool usage, procedures, or scheduling behavior. "
    "If feedback is unrelated, return the prompt exactly unchanged."
)

TRIAGE_RESPOND_UPDATE = (
    "This prompt must remain a short comma-separated list of email categories. "
    "Only add, remove, or rename list items related to classification boundaries. "
    "Global agent rules must NOT be added and must NOT leak into this prompt, "
    "including sender-ignore rules (e.g. 'emails from Alice'), signatures, "
    "writing style, tool usage, procedures, or scheduling behavior. "
    "If feedback is unrelated, return the prompt exactly unchanged."
)


In [39]:
prompts = [
    {
        "name": "main_agent",
        "prompt": store.get(("lance",), "agent_instructions").value['prompt'],
        "update_instructions": MAIN_AGENT_UPDATE,
        "when_to_update": "Update this prompt whenever there is feedback on how the agent should write emails or schedule events"
        
    },
    {
        "name": "triage-ignore", 
        "prompt": store.get(("lance",), "triage_ignore").value['prompt'],
        "update_instructions": TRIAGE_IGNORE_UPDATE,
        "when_to_update": "Only update if feedback explicitly mentions IGNORE emails. If feedback is unrelated, return the prompt exactly unchanged (same wording, spacing, punctuation)."

    },
    {
        "name": "triage-notify", 
        "prompt": store.get(("lance",), "triage_notify").value['prompt'],
        "update_instructions": TRIAGE_NOTIFY_UPDATE,
        "when_to_update": "Only update if the feedback explicitly mentions NOTIFY emails (notify me, FYI, status updates, awareness-only). If feedback is unrelated, return the prompt exactly unchanged (same wording, spacing, punctuation)."

    },
    {
        "name": "triage-respond", 
        "prompt": store.get(("lance",), "triage_respond").value['prompt'],
        "update_instructions": TRIAGE_RESPOND_UPDATE,
        "when_to_update": "Only update if the feedback explicitly mentions RESPOND emails (needs reply, draft response, reply required). Otherwise return exactly unchanged."

    },
]

In [40]:
optimizer = create_multi_prompt_optimizer(
    "openai:gpt-5-mini",
    kind="gradient",
    config={"max_reflection_steps": 2},
)

In [41]:
updated = optimizer.invoke(
    {"trajectories": conversations, "prompts": prompts}
)


In [42]:
from pprint import pprint
for m in updated:
    pprint(m)

{'name': 'main_agent',
 'prompt': "Use these tools when appropriate to help manage John's tasks "
           'efficiently. Always sign your emails `John Doe`.',
 'update_instructions': 'Make the smallest possible change to incorporate the '
                        'feedback. Do not rephrase or edit anything else. '
                        'IMPORTANT: Do NOT add or modify sender-based triage '
                        "rules such as 'ignore emails from <person>'. "
                        'Sender-based ignore rules belong ONLY in the '
                        'triage-ignore prompt. If the feedback is about '
                        'sender-based ignoring, return this prompt exactly '
                        'unchanged.',
 'when_to_update': 'Update this prompt whenever there is feedback on how the '
                   'agent should write emails or schedule events'}
{'name': 'triage-ignore',
 'prompt': 'Marketing newsletters, spam emails, mass company announcements',
 'update_instructions'

#### update the prompts in store. 

In [43]:
for i, updated_prompt in enumerate(updated):
    old_prompt = prompts[i]
    if updated_prompt['prompt'] != old_prompt['prompt']:
        name = old_prompt['name']
        print(f"updated {name}")
        if name == "main_agent":
            store.put(
                ("lance",),
                "agent_instructions",
                {"prompt":updated_prompt['prompt']}
            )
        if name == "triage-ignore":
            store.put(
                ("lance",),
                "triage_ignore",
                {"prompt":updated_prompt['prompt']}
            )
        if name == "triage-notify":
            store.put(
                ("lance",),
                "triage_notify",
                {"prompt":updated_prompt['prompt']}
            )
        if name == "triage-respond":
            store.put(
                ("lance",),
                "triage_respond",
                {"prompt":updated_prompt['prompt']}
            )

updated main_agent


In [44]:
store.get(("lance",), "agent_instructions").value['prompt']

"Use these tools when appropriate to help manage John's tasks efficiently. Always sign your emails `John Doe`."

In [45]:
response = email_agent.invoke(
    {"email_input": email_input}, 
    config=config
)

📧 Classification: RESPOND - This email requires a response


In [46]:
for m in response["messages"]:
    m.pretty_print()

================================ Human Message =================================

Respond to the email {'author': 'Alice Jones <alice.jones@bar.com>', 'to': 'John Doe <john.doe@company.com>', 'subject': 'Quick question about API documentation', 'email_thread': 'Hi John,\n\nUrgent issue - your service is down. Is there a reason why'}
================================== Ai Message ==================================
Tool Calls:
  write_email (call_qgFOPJLjlU1LMiOobvJs2jq4)
 Call ID: call_qgFOPJLjlU1LMiOobvJs2jq4
  Args:
    to: Alice Jones <alice.jones@bar.com>
    subject: Re: Quick question about API documentation
    content: Hi Alice,

Sorry to hear you’re seeing the service down — I’m escalating this to our engineering/on‑call team right away.

Could you please send any of the following details that you have; they’ll help us troubleshoot faster:
- When you first saw the failure (time and timezone)
- Any error messages, status codes, or request/trace IDs
- Which endpoint(s) and environ

In [47]:
email_input = {
    "author": "Alice Jones <alice.jones@bar.com>",
    "to": "John Doe <john.doe@company.com>",
    "subject": "Quick question about API documentation",
    "email_thread": """Hi John,

Urgent issue - your service is down. Is there a reason why""",
}

In [48]:
response = email_agent.invoke(
    {"email_input": email_input},
    config=config
)

📧 Classification: RESPOND - This email requires a response


In [49]:
conversations = [
    (
        response['messages'],
        "Ignore any emails from Alice Jones"
    )
]

In [50]:
prompts = [
    {
        "name": "main_agent",
        "prompt": store.get(("lance",), "agent_instructions").value['prompt'],
        "update_instructions": MAIN_AGENT_UPDATE,
        "when_to_update": "Update this prompt whenever there is feedback on how the agent should write emails or schedule events"
        
    },
    {
        "name": "triage-ignore", 
        "prompt": store.get(("lance",), "triage_ignore").value['prompt'],
        "update_instructions": TRIAGE_IGNORE_UPDATE,
        "when_to_update": "Only update if feedback explicitly mentions IGNORE emails. If feedback is unrelated, return the prompt exactly unchanged (same wording, spacing, punctuation)."

    },
    {
        "name": "triage-notify", 
        "prompt": store.get(("lance",), "triage_notify").value['prompt'],
        "update_instructions": TRIAGE_NOTIFY_UPDATE,
        "when_to_update": "Only update if the feedback explicitly mentions NOTIFY emails (notify me, FYI, status updates, awareness-only). If feedback is unrelated, return the prompt exactly unchanged (same wording, spacing, punctuation)."

    },
    {
        "name": "triage-respond", 
        "prompt": store.get(("lance",), "triage_respond").value['prompt'],
        "update_instructions": TRIAGE_RESPOND_UPDATE,
        "when_to_update": "Only update if the feedback explicitly mentions RESPOND emails (needs reply, draft response, reply required). Otherwise return exactly unchanged."

    },
]

In [51]:
updated = optimizer.invoke(
    {"trajectories": conversations, "prompts": prompts}
)

In [52]:
for i, updated_prompt in enumerate(updated):
    old_prompt = prompts[i]
    if updated_prompt['prompt'] != old_prompt['prompt']:
        name = old_prompt['name']
        print(f"updated {name}")
        if name == "main_agent":
            store.put(
                ("lance",),
                "agent_instructions",
                {"prompt":updated_prompt['prompt']}
            )
        if name == "triage-ignore":
            store.put(
                ("lance",),
                "triage_ignore",
                {"prompt":updated_prompt['prompt']}
            )
        if name == "triage-notify":
            store.put(
                ("lance",),
                "triage_notify",
                {"prompt":updated_prompt['prompt']}
            )
        if name == "triage-respond":
            store.put(
                ("lance",),
                "triage_respond",
                {"prompt":updated_prompt['prompt']}
            )

updated triage-ignore


In [53]:
response = email_agent.invoke(
    {"email_input": email_input},
    config=config
)

🚫 Classification: IGNORE - This email can be safely ignored


In [54]:
store.get(("lance",), "triage_ignore").value['prompt']

'Marketing newsletters, spam emails, mass company announcements, emails from Alice Jones <alice.jones@bar.com>'

## Same background prompt optimization using gpt-4.1-mini.

For prompt optimization, gpt-4.1-mini is better at:

1.Doing nothing when nothing should change

2.Respecting prompt boundaries without explicit schemas

3.Avoiding “helpful generalization”

4.Treating instructions as constraints, not suggestions

In other words, it behaves like a careful editor, not a collaborator.  

Because of this, update instructions are much simpler.

In [55]:
email_input = {
    "author": "Sam Jones <sam.jones@bar.com>",
    "to": "John Doe <john.doe@company.com>",
    "subject": "Quick question about API documentation",
    "email_thread": """Hi John,

Urgent issue - your service is down. Is there a reason why""",
}


In [56]:
response = email_agent.invoke(
    {"email_input": email_input},
    config=config
)

📧 Classification: RESPOND - This email requires a response


In [57]:
conversations = [
    (
        response["messages"],
        "Ignore any emails from Sam Jones"
    )
]


In [58]:
prompts = [
    {
        "name": "main_agent",
        "prompt": store.get(("lance",), "agent_instructions").value['prompt'],
        "update_instructions": "keep the instructions short and to the point",
        "when_to_update": "Update this prompt whenever there is feedback on how the agent should write emails or schedule events"
        
    },
    {
        "name": "triage-ignore", 
        "prompt": store.get(("lance",), "triage_ignore").value['prompt'],
        "update_instructions": "keep the instructions short and to the point",
        "when_to_update": "Update this prompt whenever there is feedback on which emails should be ignored"

    },
    {
        "name": "triage-notify", 
        "prompt": store.get(("lance",), "triage_notify").value['prompt'],
        "update_instructions": "keep the instructions short and to the point",
        "when_to_update": "Update this prompt whenever there is feedback on which emails the user should be notified of"

    },
    {
        "name": "triage-respond", 
        "prompt": store.get(("lance",), "triage_respond").value['prompt'],
        "update_instructions": "keep the instructions short and to the point",
        "when_to_update": "Update this prompt whenever there is feedback on which emails should be responded to"

    },
]

In [59]:
optimizer = create_multi_prompt_optimizer(
    "openai:gpt-4.1-mini",
    kind="prompt_memory",
)

In [60]:
updated = optimizer.invoke(
    {"trajectories": conversations, "prompts": prompts}
)

In [61]:
#json dumps is a bit easier to read
import json
print(json.dumps(updated, indent=4))

[
    {
        "name": "main_agent",
        "prompt": "Use these tools when appropriate to help manage John's tasks efficiently. Always sign your emails `John Doe`.",
        "update_instructions": "keep the instructions short and to the point",
        "when_to_update": "Update this prompt whenever there is feedback on how the agent should write emails or schedule events"
    },
    {
        "name": "triage-ignore",
        "prompt": "Marketing newsletters, spam emails, mass company announcements, emails from Alice Jones <alice.jones@bar.com>, and emails from Sam Jones <sam.jones@bar.com> should be ignored.",
        "update_instructions": "keep the instructions short and to the point",
        "when_to_update": "Update this prompt whenever there is feedback on which emails should be ignored"
    },
    {
        "name": "triage-notify",
        "prompt": "Team member out sick, build system notifications, project status updates",
        "update_instructions": "keep the instruction

In [62]:
for i, updated_prompt in enumerate(updated):
    old_prompt = prompts[i]
    if updated_prompt['prompt'] != old_prompt['prompt']:
        name = old_prompt['name']
        print(f"updated {name}")
        if name == "main_agent":
            store.put(
                ("lance",),
                "agent_instructions",
                {"prompt":updated_prompt['prompt']}
            )
        if name == "triage-ignore":
            store.put(
                ("lance",),
                "triage_ignore",
                {"prompt":updated_prompt['prompt']}
            )
        if name == "triage-notify":
            store.put(
                ("lance",),
                "triage_notify",
                {"prompt":updated_prompt['prompt']}
            )
        if name == "triage-respond":
            store.put(
                ("lance",),
                "triage_respond",
                {"prompt":updated_prompt['prompt']}
            )

updated triage-ignore


In [63]:
store.get(("lance",), "triage_ignore").value['prompt']

'Marketing newsletters, spam emails, mass company announcements, emails from Alice Jones <alice.jones@bar.com>, and emails from Sam Jones <sam.jones@bar.com> should be ignored.'

In [64]:
response = email_agent.invoke(
    {"email_input": email_input}, 
    config=config
)

🚫 Classification: IGNORE - This email can be safely ignored
